### Running bulk of multimodel testing

This is equivalent to that present in the multimodel wrapper.


In [37]:
from config.read_configurations import config_hbv as hbvArgs
from config.read_configurations import config_prms as prmsArgs
from config.read_configurations import config_sacsma as sacsmaArgs
from config.read_configurations import config_sacsma_snow as sacsmaSnowArgs
from config.read_configurations import config_hbv_hydrodl as hbvhyArgs_d


import torch
import os
import platform
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import scipy.stats
# from post import plot

from core.utils.randomseed_config import randomseed_config
from core.utils.master import create_output_dirs
from MODELS.loss_functions.get_loss_function import get_lossFun
from MODELS.test_dp_HBV_dynamic import test_dp_hbv
from core.data_processing.data_loading import loadData
from core.data_processing.normalization import transNorm
from core.utils.randomseed_config import randomseed_config
from core.data_processing.model import (
    take_sample_test,
    converting_flow_from_ft3_per_sec_to_mm_per_day
)

import warnings
warnings.filterwarnings("ignore")



# Set path to `hydro_multimodel_results` directory.
if platform.system() == 'Darwin':
    # For mac os
    out_dir = '/Users/leoglonz/Desktop/water/data/model_runs/hydro_multimodel_results'
    # Some operations are not yet working with MPS, so we might need to set some environment variables to use CPU fall instead
    # %env PYTORCH_ENABLE_MPS_FALLBACK=1

elif platform.system() == 'Windows':
    # For windows
    out_dir = 'D:\\data\\model_runs\\hydro_multimodel_results\\'

elif platform.system() == 'Linux':
    # For Colab
    out_dir = '/content/drive/MyDrive/Colab/data/model_runs/hydro_multimodel_results'

else:
    raise ValueError('Unsupported operating system.')


##-----## Multi-model Parameters ##-----##
##--------------------------------------##
# Setting dictionaries to separately manage each diff model's attributes.
models = {'HBV_dynamic': None, 'SACSMA_snow':None, 'marrmot_PRMS':None}  # 'HBV':None, 'hbvhy': None, 'SACSMA_snow':None, 'SACSMA':None,
args_list = {'HBV_dynamic': hbvhyArgs_d, 'SACSMA_snow':sacsmaSnowArgs, 'marrmot_PRMS':prmsArgs}   # 'hbvhy': hbvhyArgs, 'HBV' : hbvArgs, 'SACSMA_snow':None, 'SACSMA': sacsmaArgs,
ENSEMBLE_TYPE = 'max'  # 'median', 'avg', 'max', 'softmax'

# Load test observations and predictions from a prior run.
pred_path = os.path.join(out_dir, 'hydro_models', '671_sites_dp', 'merged_test_preds_obs', 'HBV_dynamic_SACSMA_snow_marrmot_PRMS', 'preds_multim_E50_B25_R365_BT365_H256_tr1980_1995_n16.npy')
obs_path = os.path.join(out_dir, 'hydro_models', '671_sites_dp', 'merged_test_preds_obs', 'HBV_dynamic_SACSMA_snow_marrmot_PRMS', 'obs_multim_E50_B25_R365_BT365_H256_tr1980_1995_n16.npy')
preds = np.load(pred_path, allow_pickle=True).item()
obs = np.load(obs_path, allow_pickle=True).item()

model_output = preds
y_obs = obs

In [38]:
# included in hydrodl.master.master
def calFDC(data):
    # data = Ngrid * Nday
    Ngrid, Nday = data.shape
    FDC100 = np.full([Ngrid, 100], np.nan)
    for ii in range(Ngrid):
        tempdata0 = data[ii, :]
        tempdata = tempdata0[~np.isnan(tempdata0)]
        # deal with no data case for some gages
        if len(tempdata)==0:
            tempdata = np.full(Nday, 0)
        # sort from large to small
        temp_sort = np.sort(tempdata)[::-1]
        # select 100 quantile points
        Nlen = len(tempdata)
        ind = (np.arange(100)/100*Nlen).astype(int)
        FDCflow = temp_sort[ind]
        if len(FDCflow) != 100:
            raise Exception('unknown assimilation variable')
        else:
            FDC100[ii, :] = FDCflow

    return FDC100

# included in hydrodl.post.stat
def statError(pred, target):
    ngrid, nt = pred.shape
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
    # Bias
        Bias = np.nanmean(pred - target, axis=1)
        # RMSE
        RMSE = np.sqrt(np.nanmean((pred - target)**2, axis=1))
        # ubRMSE
        predMean = np.tile(np.nanmean(pred, axis=1), (nt, 1)).transpose()
        targetMean = np.tile(np.nanmean(target, axis=1), (nt, 1)).transpose()
        predAnom = pred - predMean
        targetAnom = target - targetMean
        ubRMSE = np.sqrt(np.nanmean((predAnom - targetAnom)**2, axis=1))
        # FDC metric
        predFDC = calFDC(pred)
        targetFDC = calFDC(target)
        FDCRMSE = np.sqrt(np.nanmean((predFDC - targetFDC) ** 2, axis=1))
    # rho R2 NSE
        Corr = np.full(ngrid, np.nan)
        CorrSp = np.full(ngrid, np.nan)
        R2 = np.full(ngrid, np.nan)
        NSE = np.full(ngrid, np.nan)
        PBiaslow = np.full(ngrid, np.nan)
        PBiashigh = np.full(ngrid, np.nan)
        PBias = np.full(ngrid, np.nan)
        PBiasother = np.full(ngrid, np.nan)
        KGE = np.full(ngrid, np.nan)
        KGE12 = np.full(ngrid, np.nan)
        RMSElow = np.full(ngrid, np.nan)
        RMSEhigh = np.full(ngrid, np.nan)
        RMSEother = np.full(ngrid, np.nan)
        for k in range(0, ngrid):
            x = pred[k, :]
            y = target[k, :]
            ind = np.where(np.logical_and(~np.isnan(x), ~np.isnan(y)))[0]
            if ind.shape[0] > 0:
                xx = x[ind]
                yy = y[ind]
                # percent bias
                PBias[k] = np.sum(xx - yy) / np.sum(yy) * 100

                # FHV the peak flows bias 2%
                # FLV the low flows bias bottom 30%, log space
                pred_sort = np.sort(xx)
                target_sort = np.sort(yy)
                indexlow = round(0.3 * len(pred_sort))
                indexhigh = round(0.98 * len(pred_sort))
                lowpred = pred_sort[:indexlow]
                highpred = pred_sort[indexhigh:]
                otherpred = pred_sort[indexlow:indexhigh]
                lowtarget = target_sort[:indexlow]
                hightarget = target_sort[indexhigh:]
                othertarget = target_sort[indexlow:indexhigh]
                PBiaslow[k] = np.sum((lowpred - lowtarget)) / (np.sum(lowtarget) +0.0001)* 100
                PBiashigh[k] = np.sum(highpred - hightarget) / np.sum(hightarget) * 100
                PBiasother[k] = np.sum(otherpred - othertarget) / np.sum(othertarget) * 100
                RMSElow[k] = np.sqrt(np.nanmean((lowpred - lowtarget)**2))
                RMSEhigh[k] = np.sqrt(np.nanmean((highpred - hightarget)**2))
                RMSEother[k] = np.sqrt(np.nanmean((otherpred - othertarget)**2))

                if ind.shape[0] > 1:
                    # Theoretically at least two points for correlation
                    Corr[k] = scipy.stats.pearsonr(xx, yy)[0]
                    CorrSp[k] = scipy.stats.spearmanr(xx, yy)[0]
                    yymean = yy.mean()
                    yystd = np.std(yy)
                    xxmean = xx.mean()
                    xxstd = np.std(xx)
                    KGE[k] = 1 - np.sqrt((Corr[k]-1)**2 + (xxstd/yystd-1)**2 + (xxmean/yymean-1)**2)
                    KGE12[k] = 1 - np.sqrt((Corr[k] - 1) ** 2 + ((xxstd*yymean)/ (yystd*xxmean) - 1) ** 2 + (xxmean / yymean - 1) ** 2)
                    SST = np.sum((yy-yymean)**2)
                    SSReg = np.sum((xx-yymean)**2)
                    SSRes = np.sum((yy-xx)**2)
                    R2[k] = 1-SSRes/SST
                    NSE[k] = 1-SSRes/SST

    outDict = dict(Bias=Bias, RMSE=RMSE, ubRMSE=ubRMSE, Corr=Corr, CorrSp=CorrSp, R2=R2, NSE=NSE,
                   FLV=PBiaslow, FHV=PBiashigh, PBias=PBias, PBiasother=PBiasother, KGE=KGE, KGE12=KGE12, fdcRMSE=FDCRMSE,
                   lowRMSE=RMSElow, highRMSE=RMSEhigh, midRMSE=RMSEother)

    return outDict

In [39]:
def calculate_metrics_multi(args_list, model_outputs, y_obs_list, ensemble_type='max', out_dir=None):
    """
    Calculate stats for a multimodel ensemble.
    """
    stats_list = dict()

    for mod in args_list:
        args = args_list[mod]
        mod_out = model_outputs[mod]
        y_obs = y_obs_list[mod]

        if mod in ['SACSMA', 'SACSMA_snow', 'marrmot_PRMS', 'HBV']:
            # Note for hydrodl HBV, calculations have already been done so skip.

            # Saving data
            if out_dir:
                path = os.path.join(out_dir, 'models', '671_sites_dp', mod)
                if not os.path.exists(path):
                    os.makedirs(path, exist_ok=True)

                # Test data (obs and model results).
                for key in mod_out[0].keys():
                    if len(mod_out[0][key].shape) == 3:
                        dim = 1
                    else:
                        dim = 0
                    concatenated_tensor = torch.cat([d[key] for d in mod_out], dim=dim)
                    file_name = key + ".npy"
                    np.save(os.path.join(path, file_name), concatenated_tensor.numpy())
                    # np.save(os.path.join(args["out_dir"], args["testing_dir"], file_name), concatenated_tensor.numpy())

                # Reading and flow observations.
                print(args['target'])
                for var in args["target"]:
                    item_obs = y_obs[:, :, args["target"].index(var)]
                    file_name = var + ".npy"
                    np.save(os.path.join(path, file_name), item_obs)
                    # np.save(os.path.join(args["out_dir"], args["testing_dir"], file_name), item_obs)


            ###################### calculations here ######################
            pred_list = list()
            obs_list = list()
            flow_sim = torch.cat([d["flow_sim"] for d in mod_out], dim=1)
            flow_obs = y_obs[:, :, args["target"].index("00060_Mean")]

            pred_list.append(flow_sim.numpy())
            obs_list.append(np.expand_dims(flow_obs, 2))
            

            # we need to swap axes here to have [basin, days], and remove redundant
            # dimensions with np.squeeze().
            stats_list[mod] = [
                statError(np.swapaxes(x.squeeze(), 1, 0), np.swapaxes(y.squeeze(), 1, 0))
                for (x, y) in zip(pred_list, obs_list)
            ]
        elif mod in ['hbvhy', 'HBV_dynamic']:
            stats_list[mod] = [statError(mod_out[:,:,0], y_obs.squeeze())]
        else:
            raise ValueError(f"Unsupported model type in `models`.")

    # Calculating final statistics for the whole set of basins.
    name_list = ["flow"]
    for st, name in zip(stats_list[mod], name_list):
        count = 0
        mdstd = np.zeros([len(st), 3])
        for key in st.keys():
            # st contains the statistics on a model run like NSE and KGE.
            # Find the best result (e.g., the max, avg, median) and merge from each model.
            for i, mod in enumerate(args_list):
                if i == 0:
                    # temp contains the values of key per basin.
                    temp = stats_list[mod][0][key]
                    continue
                elif i == 1:
                    temp = np.stack((temp, stats_list[mod][0][key]), axis=1)
                else:
                    temp = np.hstack((temp, stats_list[mod][0][key].reshape(-1,1)))
            
            if len(args_list) > 1:
                if ensemble_type == 'max':
                    # print(temp, key)
                    temp = np.amax(temp, axis=1)
                    # print(temp, key)
                elif ensemble_type == 'avg':
                    temp = np.mean(temp, axis=1)
                elif ensemble_type == 'median':
                    temp = np.median(temp, axis=1)
                elif ensemble_type == 'softmax':
                    # # Softmax gets relative contributions of each model.
                    # weights = torch.nn.functional.softmax(torch.from_numpy(temp), dim=1)
                    # temp = np.sum(temp * weights.numpy(), axis=1)

                    # Instantiate weighting lstm with softmax.
                    lstm = hydroEnsemble(num_models=len(args_list), hidden_size=192, num_layers=3)
                    # Forward pass through the model
                    temp = lstm(torch.tensor(temp, dtype=torch.float))
                else:
                    raise ValueError("Invalid model ensemble type specified.")

            median = np.nanmedian(temp)  # abs(i)
            std = np.nanstd(temp)  # abs(i)
            mean = np.nanmean(temp)  # abs(i)
            k = np.array([[median, std, mean]])
            mdstd[count] = k
            count = count + 1

        # mdstd displays the statistics for each error measure in stats_list.
        mdstd = pd.DataFrame(
            mdstd, index=st.keys(), columns=["median", "STD", "mean"]
        )
        # Save the data stats from the training run:
        if out_dir and len(args_list) > 1:
            path = os.path.join(out_dir, 'multimodels', '671_sites_dp', 'n_' + ensemble_type)
            if not os.path.exists(path):
                os.makedirs(path, exist_ok=True)

            mdstd.to_csv((os.path.join(path, "mdstd_" + name + "_" + ensemble_type +".csv")))
        elif out_dir:
            path = os.path.join(out_dir, 'models', '671_sites_dp', args_list[0])
            if not os.path.exists(path):
                os.makedirs(path, exist_ok=True)

            mdstd.to_csv((os.path.join(path, "mdstd_" + name + "_" + ".csv")))
        else: continue

    # Show boxplots of the results
    # plt.rcParams["font.size"] = 14
    # keyLst = ["Bias", "RMSE", "ubRMSE", "NSE", "Corr"]
    # dataBox = list()
    # for iS in range(len(keyLst)):
    #     statStr = keyLst[iS]
    #     temp = list()
    #     # for k in range(len(st)):
    #     data = st[statStr]
    #     data = data[~np.isnan(data)]
    #     temp.append(data)
    #     dataBox.append(temp)
    # labelname = [
    #     "Hybrid differentiable model"
    # ]  # ['STA:316,batch158', 'STA:156,batch156', 'STA:1032,batch516']   # ['LSTM-34 Basin']

    # xlabel = ["Bias ($\mathregular{deg}$C)", "RMSE", "ubRMSE", "NSE", "Corr"]
    # fig = plot.plotBoxFig(
    #     dataBox, xlabel, label2=labelname, sharey=False, figsize=(16, 8)
    # )
    # fig.patch.set_facecolor("white")
    # boxPlotName = "PGML"
    # fig.suptitle(boxPlotName, fontsize=12)
    # plt.rcParams["font.size"] = 12
    # # plt.savefig(
    # #     os.path.join(args["out_dir"], args["testing_dir"], "Box_" + name + ".png")
    # # )  # , dpi=500
    # # fig.show()
    # plt.close()

    torch.cuda.empty_cache()
    print("Testing ended")

    return stats_list, mdstd


---
### Multimodel wrapper internals:
---

In [29]:
args_list = {'HBV_dynamic':hbvhyArgs_d}   # 'marrmot_PRMS':prmsArgs, 'HBV_dynamic':hbvhyArgs_d, 'hbvhy': hbvhyArgs, 'HBV' : hbvArgs, 'SACSMA_snow':None, 'SACSMA': sacsmaArgs,
ensemble_type = 'hacc' 
y_obs.keys()

dict_keys(['HBV_dynamic', 'SACSMA_snow', 'marrmot_PRMS'])

In [40]:
# Initialize
flow_preds = []
flow_obs = None
obs_trig = False

# Concatenate individual model predictions, and observation data.
for i, mod in enumerate(args_list):
    args = args_list[mod]
    mod_out = model_output[mod]
    y_ob = y_obs[mod]

    print(mod)

    if mod in ['HBV', 'SACSMA', 'SACSMA_snow', 'marrmot_PRMS']:
        # Hydro models are tested in batches, so we concatenate them and select
        # the desired flow.
        # Note: modified HBV already has this preparation done during testing.

        # Get flow predictions and swap axes to get shape [basins, days]
        pred = np.swapaxes(torch.cat([d["flow_sim"] for d in mod_out], dim=1).squeeze().numpy(), 0, 1)

        if obs_trig == False:
            # dPLHBV uses GAGES while the other hydro models use CAMELS data. This means small 
            # e-5 variation in observation data between the two. This is averaged if both models
            # are used, but to avoid double-counting data from multiply hydro models, use a trigger.
            obs = np.swapaxes(y_ob[:, :, args["target"].index("00060_Mean")].numpy(), 0, 1)
            obs_trig = True
            dup = False
        else:
            dup = True

    elif mod in ['HBV_dynamic']:
        pred = mod_out[:,:,0][:,365:] # Set dim2 = 0 to get streamflow Qr
        obs = y_ob.squeeze()[:,365:]
        dup = False

    else:
        raise ValueError(f"Unsupported model type in `models`.")
    
    if i == 0:
        tmp_pred = pred
        tmp_obs = obs
    elif i == 1:
        tmp_pred = np.stack((tmp_pred, pred), axis=2)
        if not dup:
            # Avoid double-counting GAGES obs.
            tmp_obs = np.stack((tmp_obs, obs), axis=2)
    else:
        # Combine outputs of >3 models.
        tmp_pred = np.concatenate((tmp_pred,np.expand_dims(pred, 2)), axis=2)
        if not dup:
            # Avoid double-counting GAGES obs.
            tmp_obs = np.concatenate((tmp_obs,np.expand_dims(obs, 2)), axis=2)

preds = tmp_pred
obs = tmp_obs

HBV_dynamic
SACSMA_snow
marrmot_PRMS


In [31]:
# Merge observation data.
if len(obs.shape) == 3:
    comp_obs = np.mean(obs, axis = 2)
elif len(obs.shape) == 2:
    comp_obs = obs
else:
    raise ValueError("Error reading prediction data: incorrect formatting.")


# Merge model predictions using specified merging type.
if len(preds.shape) == 3:
    if ensemble_type == 'max':
        comp_preds = np.amax(preds, axis=2)
    if ensemble_type == 'hacc':
        pass
        # comp_preds = np.amax(preds, axis=2)
    elif ensemble_type == 'min':
        comp_preds = np.amin(preds, axis=2)
    elif ensemble_type == 'avg':
        comp_preds = np.mean(preds, axis=2)
    elif ensemble_type == 'median':
        comp_preds = np.median(preds, axis=2)
    elif ensemble_type == 'softmax':
        pass
        ########### IN PROGRESS ############
        ####################################
    else:
        raise ValueError("Invalid model ensemble type specified.")
elif len(preds.shape) == 2:
    comp_preds = preds  # For single model, do nothing
else:
    raise ValueError("Error reading prediction data: incorrect formatting.")


In [32]:
# Calculating performance statistics at each basin e.g., NSE, KGE.
stat = statError(comp_preds, comp_obs)

# Testing Tadd's metrics wrapper against statError.
from dPLHydro_multimodel.metrics import Metrics
metrics = Metrics(comp_preds, comp_obs)
metrics_stats = metrics.model_dump(by_alias=True, exclude=['pred', 'target'])


mdstd = np.zeros([len(stat), 3])
count = 0

# Find median statistics across all basins.
for key in stat.keys():
    temp = stat[key]
    median = np.nanmedian(temp)
    std = np.nanstd(temp)
    mean = np.nanmean(temp)

    k = np.array([[median, std, mean]])
    mdstd[count] = k
    count = count + 1


mdstd_met = np.zeros([len(metrics_stats), 3])
count = 0

# Find median statistics across all basins.
for key in metrics_stats.keys():
    temp = metrics_stats[key]
    median = np.nanmedian(temp)
    std = np.nanstd(temp)
    mean = np.nanmean(temp)

    k = np.array([[median, std, mean]])
    mdstd_met[count] = k
    count = count + 1

    
# mdstd gives the statistics for each error measure in stats_list.
mdstd = pd.DataFrame(
    mdstd, index=stat.keys(), columns=["median", "STD", "mean"]
)
mdstd_met = pd.DataFrame(
    mdstd_met, index=metrics_stats.keys(), columns=["median", "STD", "mean"]
)
    

torch.cuda.empty_cache()
print("Testing ended")


Testing ended


In [26]:
mdstd

,median,STD,mean
Bias,0.082842,0.358185,0.063822
RMSE,1.092191,1.031007,1.299537
ubRMSE,1.062194,1.002141,1.270872
Corr,0.873422,0.113820,0.838929
CorrSp,0.880218,0.099203,0.856538
R2,0.729054,0.383030,0.629756
NSE,0.729054,0.383030,0.629756
FLV,76.345316,NaN,inf
FHV,-6.393238,25.197306,-6.333314
PBias,9.792652,28.731551,11.954376


In [33]:
mdstd

,median,STD,mean
Bias,0.082842,3.581848e-01,6.382193e-02
RMSE,1.092191,1.031007e+00,1.299537e+00
ubRMSE,1.062194,1.002141e+00,1.270872e+00
Corr,0.873422,1.138202e-01,8.389287e-01
CorrSp,0.880218,9.920339e-02,8.565381e-01
R2,0.729054,3.830304e-01,6.297559e-01
NSE,0.729054,3.830304e-01,6.297559e-01
FLV,76.345291,1.985796e+07,2.037864e+06
FHV,-6.393238,2.519731e+01,-6.333314e+00
PBias,9.792652,2.873155e+01,1.195438e+01


---
### Naive model wrapper
---

In [ ]:
import logging
from typing import Any, Optional, Tuple

import numpy as np
import numpy.typing as npt
from pydantic import BaseModel, ConfigDict
import scipy.stats as stats

log = logging.getLogger()


class Metrics(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    pred: npt.NDArray[np.float64]
    target: npt.NDArray[np.float64]
    bias: npt.NDArray[np.float64] = np.ndarray([])
    rmse: npt.NDArray[np.float64] = np.ndarray([])
    ub_rmse: npt.NDArray[np.float64] = np.ndarray([])
    fdc_rmse: npt.NDArray[np.float64] = np.ndarray([])
    corr: npt.NDArray[np.float64] = np.ndarray([])
    corr_spearman: npt.NDArray[np.float64] = np.ndarray([])
    r2: npt.NDArray[np.float64] = np.ndarray([])
    nse: npt.NDArray[np.float64] = np.ndarray([])
    pbias_low: npt.NDArray[np.float64] = np.ndarray([])
    pbias_high: npt.NDArray[np.float64] = np.ndarray([])
    pbias: npt.NDArray[np.float64] = np.ndarray([])
    pbias_mid: npt.NDArray[np.float64] = np.ndarray([])
    kge: npt.NDArray[np.float64] = np.ndarray([])
    kge_12: npt.NDArray[np.float64] = np.ndarray([])
    rmse_low: npt.NDArray[np.float64] = np.ndarray([])
    rmse_high: npt.NDArray[np.float64] = np.ndarray([])
    rmse_mid: npt.NDArray[np.float64] = np.ndarray([])
    
    
    def __init__(self, pred: npt.NDArray[np.float64], target: npt.NDArray[np.float64]):
        super(Metrics, self).__init__(pred=pred, target=target)

    def model_post_init(self, __context: Any):
        self.bias = self._bias(self.pred, self.target)
        self.rmse = self._rmse(self.pred, self.target)
        
        pred_mean = self.tile_mean(self.pred)
        target_mean = self.tile_mean(self.target)        
        pred_anom = self.pred - pred_mean
        target_anom = self.target - target_mean
        self.ub_rmse = self._rmse(pred_anom, target_anom)
        
        pred_fdc = self._calc_fdc(self.pred)
        target_fdc = self._calc_fdc(self.target)
        self.fdc_rmse = self._rmse(pred_fdc, target_fdc)
        
        self.corr = np.full(self.ngrid, np.nan)
        self.corr_spearman = np.full(self.ngrid, np.nan)
        self.r2 = np.full(self.ngrid, np.nan)
        self.nse = np.full(self.ngrid, np.nan)
        self.pbias_low = np.full(self.ngrid, np.nan)
        self.pbias_high = np.full(self.ngrid, np.nan)
        self.pbias = np.full(self.ngrid, np.nan)
        self.pbias_mid = np.full(self.ngrid, np.nan)
        self.kge = np.full(self.ngrid, np.nan)
        self.kge_12 = np.full(self.ngrid, np.nan)
        self.rmse_low = np.full(self.ngrid, np.nan)
        self.rmse_high = np.full(self.ngrid, np.nan)
        self.rmse_mid = np.full(self.ngrid, np.nan)
        for i in range(0, self.ngrid):
            _pred = self.pred[i]
            _target = self.target[i]
            non_nan_idx = np.where(
                np.logical_and(~np.isnan(_pred), ~np.isnan(_target))
            )[0]
            if non_nan_idx.shape[0] > 0:
                pred = _pred[non_nan_idx]
                target = _target[non_nan_idx]

                pred_sort = np.sort(pred)
                target_sort = np.sort(target)
                index_low = round(0.3 * pred_sort.shape[0])
                index_high = round(0.98 * pred_sort.shape[0])
                low_pred = pred_sort[:index_low]
                high_pred = pred_sort[index_high:]
                mid_pred = pred_sort[index_low:index_high]
                low_target = target_sort[:index_low]
                high_target = target_sort[index_high:]
                mid_target = target_sort[index_low:index_high]

                self.pbias[i] = self._p_bias(pred, target)
                self.pbias_low[i] = self._p_bias(low_pred, low_target)
                self.pbias_high[i] = self._p_bias(high_pred, high_target)
                self.pbias_mid[i] = self._p_bias(mid_pred, mid_target)
                self.rmse_low[i] = self._rmse(low_pred, low_target, axis=0)
                self.rmse_high[i] = self._rmse(high_pred, high_target, axis=0)
                self.rmse_mid[i] = self._rmse(mid_pred, mid_target, axis=0)

                if non_nan_idx.shape[0] > 1:
                    self.corr[i] = self._corr(pred, target)
                    self.corr_spearman[i] = self._corr_spearman(pred, target)
                    _pred_mean = pred.mean()
                    _target_mean = target.mean()
                    _pred_std = np.std(pred)
                    _target_std = np.std(target)
                    self.kge[i] = self._kge(
                        _pred_mean, _target_mean, _pred_std, _target_std, self.corr[i]
                    )
                    self.kge_12[i] = self._kge_12(
                        _pred_mean, _target_mean, _pred_std, _target_std, self.corr[i]
                    )
                    self.nse[i], self.r2[i] = self._nse_r2(pred, target, _target_mean)

    def _calc_fdc(self, data: npt.NDArray[np.float64]) -> npt.NDArray[np.float64]:
        """
        Calculate flow duration curve for each grid
        """
        fdc_100 = np.full([self.ngrid, 100], np.nan)
        for i in range(self.ngrid):
            data_slice = data[i]
            non_nan_data_slice = data_slice[~np.isnan(data_slice)]
            if len(non_nan_data_slice) == 0:
                non_nan_data_slice = np.full(self.nt, 0)
            sorted_data = np.sort(non_nan_data_slice)[::-1]
            Nlen = len(non_nan_data_slice)
            ind = (np.arange(100) / 100 * Nlen).astype(int)
            fdc_flow = sorted_data[ind]
            if len(fdc_flow) != 100:
                raise Exception("unknown assimilation variable")
            else:
                fdc_100[i] = fdc_flow
        return fdc_100

    @property
    def ngrid(self) -> int:
        """
        Calculate number of grids
        """
        return self.pred.shape[0]

    @property
    def nt(self) -> int:
        """
        Calculate number of time steps
        """
        return self.pred.shape[1]
    
    def tile_mean(self, data: npt.NDArray[np.float64]) -> npt.NDArray[np.float64]:
        """
        Calculate mean of target
        """
        return np.tile(np.nanmean(data, axis=1), (self.nt, 1)).transpose()

    @staticmethod
    def _rmse(
        pred: npt.NDArray[np.float64],
        target: npt.NDArray[np.float64],
        axis: Optional[int] = 1,
    ) -> npt.NDArray[np.float64]:
        """
        Calculate root mean square error
        """
        return np.sqrt(np.nanmean((pred - target) ** 2, axis=axis))
    
    @staticmethod
    def _bias(pred: npt.NDArray[np.float64], target: npt.NDArray[np.float64],) -> npt.NDArray[np.float64]:
        """
        Calculate bias
        """
        return np.nanmean((pred - target), axis=1)

    @staticmethod
    def _p_bias(
        pred: npt.NDArray[np.float64], target: npt.NDArray[np.float64]
    ) -> np.float64:
        """
        Calculate p bias
        """
        p_bias = np.sum(pred - target) / np.sum(target) * 100
        return p_bias

    @staticmethod
    def _corr(
        pred: npt.NDArray[np.float64], target: npt.NDArray[np.float64]
    ) -> npt.NDArray[np.float64]:
        """
        Calculate correlation
        """
        corr = stats.pearsonr(pred, target)[0]
        return corr

    @staticmethod
    def _corr_spearman(
        pred: npt.NDArray[np.float64], target: npt.NDArray[np.float64]
    ) -> np.float64:
        """
        Calculate spearman r
        """
        corr_spearman = stats.spearmanr(pred, target)[0]
        return corr_spearman

    @staticmethod
    def _kge(
        pred_mean: np.float64,
        target_mean: np.float64,
        pred_std: np.float64,
        target_std: np.float64,
        corr: np.float64,
    ) -> npt.NDArray[np.float64]:
        """
        Calculate KGE
        """
        kge = 1 - np.sqrt(
            (corr - 1) ** 2
            + (pred_std / target_std - 1) ** 2
            + (pred_mean / target_mean - 1) ** 2
        )
        return kge

    @staticmethod
    def _kge_12(
        pred_mean: np.float64,
        target_mean: np.float64,
        pred_std: np.float64,
        target_std: np.float64,
        corr: np.float64,
    ) -> npt.NDArray[np.float64]:
        """
        Calculate KGE 1-2
        """
        kge_12 = 1 - np.sqrt(
            (corr - 1) ** 2
            + ((pred_std * target_mean) / (target_std * pred_mean) - 1) ** 2
            + (pred_mean / target_mean - 1) ** 2
        )
        return kge_12

    @staticmethod
    def _nse_r2(
        pred: npt.NDArray[np.float64],
        target: npt.NDArray[np.float64],
        target_mean: np.float64,
    ) -> Tuple[np.float64, np.float64]:
        """
        Calculate NSE/R2
        """
        sst = np.sum((target - target_mean) ** 2)
        ssres = np.sum((target - pred) ** 2)
        r2 = 1 - ssres / sst
        nse = 1 - ssres / sst
        return nse, r2